### 과제 2: 배송 성과 분석
**과제 2: 배송 성과 분석 및 지역별 물류 최적화 방안**
주문부터 배송 완료까지의 시간을 분석하고, 지역별 배송 성과 차이를 파악하여 물류 최적화 방안을 도출하세요. 배송 지연이 고객 만족도(리뷰 점수)에 미치는 영향도 함께 분석하세요



## EDA 진행 
**목표:** 지역별 배송 시간의 평균과 분포를 확인하고, 배송 시간과 고객 리뷰 점수 간의 관계를 파악합니다.

### **분석 로직:**

1. **날짜/시간 변환**: `orders` 데이터셋의 `order_purchase_timestamp`와 `order_delivered_customer_date`를 `datetime` 타입으로 변환합니다.
2. **배송 시간 계산**: `order_delivered_customer_date`에서 `order_purchase_timestamp`를 빼서 배송 소요 시간을 계산합니다.
3. **데이터 병합**: `orders`와 `geolocation` 데이터셋을 병합하여 주문별 지역 정보를 추가합니다. 또한, `orders`와 `reviews` 데이터셋을 병합하여 주문별 리뷰 점수를 연결합니다.
4. **지역별 분석**: `customer_state`를 기준으로 그룹화하여 지역별 평균 배송 시간을 계산합니다.
5. **상관관계 분석**: 배송 소요 시간과 `review_score` 간의 상관계수를 계산합니다.

### **시각화 계획:**

- **박스플롯**: 지역별(주/도시) 배송 시간의 분포를 한눈에 비교하기 위해 박스플롯을 그립니다.
- **산점도**: 배송 소요 시간과 리뷰 점수 간의 관계를 시각적으로 확인하기 위해 산점도를 그립니다.
- **히스토그램**: 리뷰 점수별(1~5점) 배송 소요 시간의 분포를 비교하는 히스토그램을 그립니다.


오늘 진행할 내용은  
**과제 2: 배송 성과 분석 및 지역별 물류 최적화 방안**
주문부터 배송 완료까지의 시간을 분석하고, 지역별 배송 성과 차이를 파악하여 물류 최적화 방안을 도출하세요. 배송 지연이 고객 만족도(리뷰 점수)에 미치는 영향도 함께 분석 




# 데이터 전처리 코드 

In [ ]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np

# 시각화 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns
# 폰트 깨짐 방지: Matplotlib에 한글 폰트 설정
plt.rc('font', family='AppleGothic')
# 마이너스 기호 깨짐 방지
plt.rc('axes', unicode_minus=False)
# 경고 무시
import warnings
warnings.filterwarnings('ignore')

# 주요 테이블 로딩

customers = pd.read_csv("olist_customers_dataset.csv")
orders = pd.read_csv("olist_orders_dataset.csv")
order_items = pd.read_csv("olist_order_items_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")
sellers = pd.read_csv("olist_sellers_dataset.csv")
payments = pd.read_csv("olist_order_payments_dataset.csv")
reviews = pd.read_csv("olist_order_reviews_dataset.csv")
category_name = pd.read_csv("product_category_name_translation.csv")
geolocation= pd.read_csv("olist_geolocation_dataset.csv")


### 데이터 형 변환 (포르투갈어-> 영어, date_time 형식으로 변환)

In [ ]:
# 각 데이터셋 크기 확인
datasets = {
    "customers": customers,
    "orders": orders,
    "order_items": order_items,
    "products": products,
    "sellers": sellers,
    "payments": payments,
    "reviews": reviews,
    "category_name": category_name,
    "geolocation": geolocation
}

for name, df in datasets.items():
    print(f"{name}: {df.shape}")
    display(df.head(2))  # 앞부분 2행만 미리보기


# ======================
# [1] Products 카테고리명 번역 (포르투갈어 → 영어)
# ======================
products = pd.read_csv("olist_products_dataset.csv")
category_translation = pd.read_csv("product_category_name_translation.csv")

# 병합 후 컬럼 정리
products = (
    products.merge(category_translation, on="product_category_name", how="left")
            .drop("product_category_name", axis=1)
            .rename(columns={"product_category_name_english": "product_category_name"})
)

print("\n=== 변환 후 'products' 카테고리 예시 ===")
print(products["product_category_name"].head())


# ======================
# [2] Orders 날짜 컬럼 변환
# ======================
datetime_cols = [
    "order_purchase_timestamp",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]
# orders 데이터셋의 모든 날짜/시간 관련 컬럼을 datetime 타입으로 변환
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'])
orders['order_delivered_carrier_date'] = pd.to_datetime(orders['order_delivered_carrier_date'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'])



# ======================
# [3] Reviews 날짜 컬럼 변환
# ======================
reviews[["review_creation_date", "review_answer_timestamp"]] = reviews[
    ["review_creation_date", "review_answer_timestamp"]
].apply(pd.to_datetime, errors="coerce")

print("\n=== Reviews 날짜 컬럼 dtype 확인 ===")
print(reviews[["review_creation_date", "review_answer_timestamp"]].dtypes)




### 결측치 처리

- orders: 배송 완료일(order_delivered_customer_date) 없는 주문 제거 ✅
- orders: 결제 승인일(order_approved_at) 없는 주문 제거 ✅
- reviews: 코멘트 결측치 → "no comment" 대체 ✅

In [ ]:
# orders 데이터셋을 불러옵니다.
# 사용자 정의 변수명을 참조하여 코드를 작성합니다.
print("=== 'orders' 데이터셋 결측치 현황 ===")
print(orders.isnull().sum())

# 배송 완료일(`order_delivered_customer_date`)이 없는 주문은 배송이 완료되지 않은 상태입니다.
# 배송 성과 분석을 위해 이 행들을 제거합니다.
orders.dropna(subset=['order_delivered_customer_date'], inplace=True)
print("\n배송 미완료 주문 제거 후 데이터 크기:", orders.shape)

# 추가적으로, 결제 승인일(`order_approved_at`)이 없는 주문도 분석에서 제외할 수 있습니다.
# 이는 유효하지 않은 주문으로 간주될 수 있기 때문입니다.
orders.dropna(subset=['order_approved_at'], inplace=True)
print("\n결제 미승인 주문 제거 후 최종 데이터 크기:", orders.shape)
print("\n결제 미승인 주문 제거 후 최종 데이터 크기:", orders .shape)


# 결측치가 제거된 최종 데이터셋의 결측치 현황을 다시 확인합니다.
print("\n=== 최종 데이터셋 결측치 재확인 ===")
print(orders.isnull().sum())

# 결측치가 제거된 데이터셋의 상위 5개 행을 확인합니다.
print("\n=== 최종 데이터셋 미리보기 ===")
print(orders.head())

# reviews 데이터셋을 불러옵니다.
reviews = pd.read_csv('olist_order_reviews_dataset.csv')

print("=== 'olist_order_reviews_dataset.csv' 결측치 현황 ===")
print(reviews.isnull().sum())

# 코멘트 관련 컬럼(`review_comment_title`, `review_comment_message`)의 결측치는
# 고객이 코멘트를 남기지 않았음을 의미하므로, 'no comment'로 채울 수 있습니다.
reviews['review_comment_title'].fillna('no comment', inplace=True)
reviews['review_comment_message'].fillna('no comment', inplace=True)

print("\n결측치 처리 후 현황:")
print(reviews.isnull().sum())

## 이상값 처리

- 이상치 처리
    - orders: 배송 소요일(delivery_time_days) IQR 기반 제거 ✅
    - order_items: 가격(price) IQR 기반 제거 ✅
    - products: 무게(product_weight_g) IQR 기반 제거 ✅
    - payments: 결제 금액(payment_value) IQR 기반 제거 ✅

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터셋 불러오기
orders = pd.read_csv("olist_orders_dataset.csv")
order_items = pd.read_csv("olist_order_items_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")
payments = pd.read_csv("olist_order_payments_dataset.csv")

# 폰트 깨짐 방지: Matplotlib에 한글 폰트 설정
plt.rc('font', family='AppleGothic')
# 마이너스 기호 깨짐 방지
plt.rc('axes', unicode_minus=False)

# orders 데이터 날짜 변환 및 배송 소요 시간 계산
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders.dropna(subset=['order_delivered_customer_date'], inplace=True)
orders['delivery_time_days'] = (orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']).dt.total_seconds() / (24 * 3600)

# 시각화를 위한 원본 데이터 복사
orders_original = orders.copy()
order_items_original = order_items.copy()
products_original = products.copy()
payments_original = payments.copy()

# =================================================================
# 이상값 처리 및 시각화
# =================================================================

fig, axes = plt.subplots(2, 4, figsize=(15, 10))
fig.suptitle('이상값 처리 전후 박스플롯 비교', fontsize=16)

# ----------------- 배송 소요 시간 (orders) -----------------
# 처리 전
sns.boxplot(x=orders_original['delivery_time_days'], ax=axes[0, 0])
axes[0, 0].set_title('배송 소요 시간 (처리 전)')
axes[0, 0].set_xlabel('일수')

# 이상값 처리
Q1 = orders['delivery_time_days'].quantile(0.25)
Q3 = orders['delivery_time_days'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
orders_cleaned = orders[(orders['delivery_time_days'] >= lower_bound) & (orders['delivery_time_days'] <= upper_bound)]

# 처리 후
sns.boxplot(x=orders_cleaned['delivery_time_days'], ax=axes[1, 0])
axes[1, 0].set_title('배송 소요 시간 (처리 후)')
axes[1, 0].set_xlabel('일수')

# ----------------- 상품 가격 (order_items) -----------------
# 처리 전
sns.boxplot(x=order_items_original['price'], ax=axes[0, 1])
axes[0, 1].set_title('상품 가격 (처리 전)')
axes[0, 1].set_xlabel('가격 (R$)')

# 이상값 처리
Q1 = order_items['price'].quantile(0.25)
Q3 = order_items['price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
order_items_cleaned = order_items[(order_items['price'] >= lower_bound) & (order_items['price'] <= upper_bound)]

# 처리 후
sns.boxplot(x=order_items_cleaned['price'], ax=axes[1, 1])
axes[1, 1].set_title('상품 가격 (처리 후)')
axes[1, 1].set_xlabel('가격 (R$)')

# ----------------- 상품 무게 (products) -----------------
# 처리 전
sns.boxplot(x=products_original['product_weight_g'], ax=axes[0, 2])
axes[0, 2].set_title('상품 무게 (처리 전)')
axes[0, 2].set_xlabel('무게 (g)')

# 이상값 처리
Q1 = products['product_weight_g'].quantile(0.25)
Q3 = products['product_weight_g'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
products_cleaned = products[(products['product_weight_g'] >= lower_bound) & (products['product_weight_g'] <= upper_bound)]

# 처리 후
sns.boxplot(x=products_cleaned['product_weight_g'], ax=axes[1, 2])
axes[1, 2].set_title('상품 무게 (처리 후)')
axes[1, 2].set_xlabel('무게 (g)')

# ----------------- 결제 금액 (payments) -----------------
# 처리 전
sns.boxplot(x=payments_original['payment_value'], ax=axes[0, 3])
axes[0, 3].set_title('결제 금액 (처리 전)')
axes[0, 3].set_xlabel('금액 (R$)')

# 이상값 처리
Q1 = payments['payment_value'].quantile(0.25)
Q3 = payments['payment_value'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
payments_cleaned = payments[(payments['payment_value'] >= lower_bound) & (payments['payment_value'] <= upper_bound)]

# 처리 후
sns.boxplot(x=payments_cleaned['payment_value'], ax=axes[1, 3])
axes[1, 3].set_title('결제 금액 (처리 후)')
axes[1, 3].set_xlabel('금액 (R$)')

# 전체 레이아웃 조정
plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()



### 데이터 병합 merge 및 중복값 제거 

In [ ]:

# orders와 customers 병합 (customer_id 기준)
df_merged = pd.merge(orders, customers, on='customer_id', how='left')

# df_merged와 order_items 병합 (order_id 기준)
df_merged = pd.merge(df_merged, order_items, on='order_id', how='left')

# df_merged와 payments 병합 (order_id 기준)
df_merged = pd.merge(df_merged, payments, on='order_id', how='left')

# df_merged와 reviews 병합 (order_id 기준)
df_merged = pd.merge(df_merged, reviews, on='order_id', how='left')

# df_merged와 products 병합 (product_id 기준)
df_merged = pd.merge(df_merged, products, on='product_id', how='left')

# df_merged와 sellers 병합 (seller_id 기준)
df_merged = pd.merge(df_merged, sellers, on='seller_id', how='left')

# df_merged와 category_name 병합 (product_category_name 기준)
df_merged = pd.merge(df_merged, category_name, on='product_category_name', how='left')

# geolocation 데이터는 다대다 관계이므로, 추후에 필요에 따라 병합
# 현재는 geolocation_zip_code_prefix를 기준으로 고객 및 판매자 지역을 분석

# 최종 데이터프레임 미리보기
print("\n\n=== 모든 데이터셋 병합 후 최종 데이터프레임 ===")
print("데이터 크기:", df_merged.shape)
print(df_merged.head())

# 모든 컬럼이 동일한 중복 행을 제거합니다.
# 특히 여러 데이터셋을 병합하는 과정에서 중복이 발생할 수 있습니다.
initial_rows = df_merged.shape[0]
df_merged.drop_duplicates(inplace=True)
cleaned_rows = df_merged.shape[0]

print("\n=== 중복값 처리 현황 ===")
print(f"처리 전 행 수: {initial_rows}")
print(f"중복 제거 후 행 수: {cleaned_rows}")
print(f"제거된 중복 행 수: {initial_rows - cleaned_rows}")


### 추가 파생 변수 생성 

In [ ]:
df_merged['order_purchase_timestamp'] = pd.to_datetime(df_merged['order_purchase_timestamp'])
df_merged['order_delivered_customer_date'] = pd.to_datetime(df_merged['order_delivered_customer_date'])
df_merged['order_estimated_delivery_date'] = pd.to_datetime(df_merged['order_estimated_delivery_date'])

#추가 파생 변수 생성 
# 배송 소요 시간 (delivery_time_days)
df_merged['delivery_time_days'] = (df_merged['order_delivered_customer_date'] - df_merged['order_purchase_timestamp']).dt.total_seconds() / (24 * 3600)

# 배송 지연 여부 (delivery_delay)
df_merged['delivery_delay'] = (df_merged['order_delivered_customer_date'] - df_merged['order_estimated_delivery_date']).dt.total_seconds() / (24 * 3600)

# 구매 시간 관련 변수
df_merged['order_purchase_year'] = df_merged['order_purchase_timestamp'].dt.year
df_merged['order_purchase_month'] = df_merged['order_purchase_timestamp'].dt.month
df_merged['order_purchase_dayofweek'] = df_merged['order_purchase_timestamp'].dt.dayofweek
df_merged['order_purchase_hour'] = df_merged['order_purchase_timestamp'].dt.hour



df_merged.to_csv("olist_master_clean.csv", index=False)



***코드 진행 내용 요약***

제출해주신 코드는 크게 네 가지 단계로 진행되었습니다.

1. 데이터 불러오기 및 결측치 확인: 필요한 라이브러리를 임포트하고 9개의 CSV 파일을 로드했습니다. orders 데이터셋의 결측치를 확인하고, 배송 및 결제 관련 결측 행을 제거했습니다. reviews 데이터셋의 코멘트 결측치는 'no comment'로 채웠습니다.

2. 이상값 처리 및 시각화: orders (delivery_time_days), order_items (price), products (product_weight_g), payments (payment_value) 데이터셋의 주요 수치형 변수들에 대해 IQR(사분위 범위)을 활용한 이상값 처리를 진행했습니다. 이상값 처리 전후의 변화를 2행 4열의 박스플롯으로 시각화하여 한눈에 비교할 수 있도록 했습니다.

3. 데이터 병합: customers, orders, order_items, payments 데이터셋을 순차적으로 병합하여 하나의 통합된 데이터프레임(df_merged)을 만들었습니다. 이 과정에서 order_id에 결측치가 있는 행을 제거했습니다.

4. 파생 변수 생성 및 저장: 병합된 데이터프레임에서 delivery_days, delivery_delay, order_year, order_month, order_dayofweek와 같은 새로운 파생 변수들을 생성했습니다. 마지막으로 이 데이터를 olist_master_clean.csv 파일로 저장했습니다.



###  과제 2: 배송 성과 분석 및 지역별 물류 최적화 방안

주문부터 배송 완료까지의 시간을 분석하고, 지역별 배송 성과 차이를 파악하여 물류 최적화 방안을 도출하세요. 배송 지연이 고객 만족도(리뷰 점수)에 미치는 영향도 함께 분석하세요.

### 1. 배송 소요일 계산과 배송지연 파생 변수 생성 
    
    •	order_purchase_timestamp : 고객이 주문한 시각
	•	order_delivered_customer_date : 실제 고객에게 배송 완료된 시각
	•	order_estimated_delivery_date : 시스템이 예측한 배송 완료 예정일

👉 여기서 배송 소요일(delivery_days)은 order_delivered_customer_date - order_purchase_timestamp
👉 배송 지연 여부(is_late)는 order_delivered_customer_date > order_estimated_delivery_date

In [ ]:
# 필요한 컬럼이 datetime인지 확인 후 변환
df_merged["order_purchase_timestamp"] = pd.to_datetime(df_merged["order_purchase_timestamp"], errors="coerce")
df_merged["order_delivered_customer_date"] = pd.to_datetime(df_merged["order_delivered_customer_date"], errors="coerce")
df_merged["order_estimated_delivery_date"] = pd.to_datetime(df_merged["order_estimated_delivery_date"], errors="coerce")

# 배송 소요일 계산 (일 단위)
df_merged["delivery_days"] = (df_merged["order_delivered_customer_date"] - df_merged["order_purchase_timestamp"]).dt.days

# 배송 지연 여부 (지연이면 1, 제때면 0)
df_merged["is_late"] = (df_merged["order_delivered_customer_date"] > df_merged["order_estimated_delivery_date"]).astype(int)

# 배송 관련 결측치 제거 (배송 완료일 없는 경우 제외)
df_delivery = df_merged.dropna(subset=["order_delivered_customer_date", "order_estimated_delivery_date"]).copy()

# 결과 확인
print("\n=== 배송 성과 관련 데이터 미리보기 ===")
print(df_delivery[["order_id", "order_purchase_timestamp", "order_delivered_customer_date", 
                   "order_estimated_delivery_date", "delivery_days", "is_late"]].head(10))

# 지연 비율 출력
late_ratio = df_delivery["is_late"].mean()
print(f"\n배송 지연 비율: {late_ratio:.2%}")

### 2. 지역별 배송 성과 분석 
 customer_state 기준으로 집계
- 평균 배송 소요일
-  배송 지연 비율
- 주문 건수


In [ ]:
# 지역별 배송 성과 집계
region_delivery = df_delivery.groupby("customer_state").agg(
    avg_delivery_days=("delivery_days", "mean"),
    late_ratio=("is_late", "mean"),
    order_count=("order_id", "count")
).reset_index() 

# 배송 지연 비율(%)로 변환
region_delivery["late_ratio"] = region_delivery["late_ratio"] * 100

# 평균 배송일 기준 정렬
region_delivery = region_delivery.sort_values(by="avg_delivery_days", ascending=True)

print("\n=== 지역별 배송 성과 요약 ===")
print(region_delivery.head(10))  # 상위 10개만 확인

# 시각화 (지역별 평균 배송 소요일 & 지연율)
import matplotlib.pyplot as plt

fig, ax1 = plt.subplots(figsize=(14, 6))

# 평균 배송일 막대 그래프
ax1.bar(region_delivery["customer_state"], region_delivery["avg_delivery_days"], color="skyblue", label="평균 배송일")
ax1.set_ylabel("평균 배송일 (일)", fontsize=12)
ax1.set_xlabel("지역 (State)", fontsize=12)
ax1.tick_params(axis='x', rotation=45)

# 지연율 라인 그래프 (2축)
ax2 = ax1.twinx()
ax2.plot(region_delivery["customer_state"], region_delivery["late_ratio"], color="red", marker="o", label="배송 지연율")
ax2.set_ylabel("배송 지연율 (%)", fontsize=12)

# 범례
fig.legend(loc="upper right", bbox_to_anchor=(0.9, 0.9))

plt.title("지역별 배송 성과 (평균 배송일 & 지연율)", fontsize=14)
plt.tight_layout()
plt.show()

## 지역별 배송 성과 비교 단계 물류 효율성 차이 시각화 
- 상위/ 히위 성과 지역을 비교해 물류효율성 차이 시각화 
1. 지역별 핵심 지표 계산
	•	평균 배송일 (delivery_days)
	•	지연률 (is_late 비율)
	•	평균 리뷰 점수 (review_score)

In [ ]:
region_perf = customers.merge(
    df_delivery[['order_id','delivery_days','is_late']], 
    on='order_id'
)
region_perf = region_perf.merge(
    reviews[['order_id','review_score']], 
    on='order_id'
)

# 지역별 통계
region_summary = region_perf.groupby('customer_state').agg(
    avg_delivery=('delivery_days','mean'),
    late_rate=('is_late','mean'),
    avg_review=('review_score','mean'),
    order_count=('order_id','count')
).reset_index()

region_summary = region_summary.sort_values(by='avg_delivery')

print(df_delivery)


In [154]:

print(df_delivery.head())

                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
2  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
3  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
4  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
2    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
3    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
4    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 21:25:13   
1          2017-10-04 19:55:00           2017-10

In [ ]:
# 상위 / 하위 지역 추출 
# 배송 성과 기준 (예: 평균 배송일 짧은 = 상위)
top_regions = region_summary.nsmallest(5, 'avg_delivery')
bottom_regions = region_summary.nlargest(5, 'avg_delivery')

## 시각화 예시 

### 예상 결과 
- 상위 지역 -> 배송일 짧고 지연률 낮으며 리뷰 점수 높음 --> 물류 체계 안정적
- 하위 지역 -> 배송일 길고 지연률 높으며 리뷰 점수 낮음 -> 물류 거점 확충, 배송 경로 최적화 필요 

### 1.  평균 배송일 비교(막대 그래프)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.bar(top_regions['customer_state'], top_regions['avg_delivery'], label="Top (빠른 배송)")
plt.bar(bottom_regions['customer_state'], bottom_regions['avg_delivery'], label="Bottom (느린 배송)")
plt.ylabel("평균 배송일")
plt.xlabel("지역(State)")
plt.title("지역별 평균 배송일 상위/하위 비교")
plt.legend()
plt.show()

### 2. 지연률 + 리뷰 점수 함께 시각화 

In [ ]:
fig, ax1 = plt.subplots(figsize=(12,6))

ax2 = ax1.twinx()

ax1.bar(region_summary['customer_state'], region_summary['late_rate'], alpha=0.6, label='지연률')
ax2.plot(region_summary['customer_state'], region_summary['avg_review'], color='red', marker='o', label='평균 리뷰 점수')

ax1.set_ylabel("지연률")
ax2.set_ylabel("평균 리뷰 점수")
plt.title("지역별 배송 지연률 vs 리뷰 점수")
plt.show()

### 3. 리뷰 점수별 평균 배송일과 지연율 
- 이제 배송 성과(delivery_days, is_late)가 **리뷰 점수(review_score)**에 어떤 영향을 주는지 분석하는 코드

In [ ]:
# 리뷰 점수별 배송 성과 집계
review_delivery = df_delivery.groupby("review_score").agg(
    avg_delivery_days=("delivery_days", "mean"),
    late_ratio=("is_late", "mean"),
    review_count=("order_id", "count")
).reset_index()

# 지연율 % 변환
review_delivery["late_ratio"] = review_delivery["late_ratio"] * 100

print("\n=== 리뷰 점수별 배송 성과 요약 ===")
print(review_delivery)

# 시각화
fig, ax1 = plt.subplots(figsize=(10, 5))

# 평균 배송일 막대 그래프
ax1.bar(review_delivery["review_score"], review_delivery["avg_delivery_days"], 
        color="lightgreen", label="평균 배송일")
ax1.set_xlabel("리뷰 점수", fontsize=12)
ax1.set_ylabel("평균 배송일 (일)", fontsize=12)

# 지연율 라인 그래프
ax2 = ax1.twinx()
ax2.plot(review_delivery["review_score"], review_delivery["late_ratio"], 
         color="red", marker="o", label="배송 지연율")
ax2.set_ylabel("배송 지연율 (%)", fontsize=12)

# 범례
fig.legend(loc="upper right", bbox_to_anchor=(0.9, 0.9))
plt.title("리뷰 점수별 배송 성과 (평균 배송일 & 지연율)", fontsize=14)
plt.tight_layout()
plt.show()

### 4. 배송 지연여부와 리뷰 점수 분포 
 - 📊 실행하면 확인할 수 있는 것:
	1.	**리뷰 점수 낮은 그룹(1~2점)**은 평균 배송일이 길고 지연율이 높을 가능성이 큼
	2.	**리뷰 점수 높은 그룹(4~5점)**은 평균 배송일이 짧고 지연율이 낮음
	3.	is_late 여부에 따라 리뷰 점수 분포 차이가 극명하게 나타남


In [ ]:
# 지연 여부별 리뷰 점수 분포
late_review = df_delivery.groupby("is_late")["review_score"].value_counts(normalize=True).unstack().fillna(0)

print("\n=== 지연 여부에 따른 리뷰 점수 분포 (%) ===")
print((late_review * 100).round(2))

# 시각화
late_review.T.plot(kind="bar", figsize=(10,6))
plt.title("배송 지연 여부에 따른 리뷰 점수 분포", fontsize=14)
plt.xlabel("리뷰 점수")
plt.ylabel("비율 (%)")
plt.legend(["정시 배송", "지연 배송"])
plt.xticks(rotation=0)
plt.show()

### 상관계수 확인 과 단순 회귀 분석 
- 리뷰 점수와 배송소요시간,지연 여부는 음의 상관관계를 가질 것이다.
이렇게 하면
	1.	상관분석 → 단순 관계 확인
	2.	선형 회귀 → 배송일 ↔ 리뷰 점수의 정량적 영향
	3.	로지스틱 회귀 → 배송 지연이 긍정/부정 리뷰에 미치는 영향

In [ ]:
# 숫자형 변수만 추출
corr_data = df_delivery[["delivery_days", "is_late", "review_score"]]

# 상관계수 계산
corr = corr_data.corr()

print("\n=== 배송 성과와 리뷰 점수 간 상관계수 ===")
print(corr)

# 히트맵 시각화
plt.figure(figsize=(6,4))
sns.heatmap(corr, annot=True, cmap="RdYlGn", center=0, fmt=".2f")
plt.title("배송 성과 ↔ 리뷰 점수 상관관계")
plt.show()



✅ 해석
	•	delivery_days ↔ review_score: 음의 상관(-0.32) → 배송일이 길수록 리뷰 점수가 떨어짐
	•	is_late ↔ review_score: 더 강한 음의 상관(-0.45) → 지연 여부가 점수 하락에 큰 영향
	•	delivery_days ↔ is_late: 양의 상관(0.65) → 당연히 배송일이 길면 지연 확률도 높아짐

In [ ]:
%pip install statsmodels

## 단순 회귀 분석 
•	coef가 음수이면 → 배송일이 길수록 리뷰 점수가 낮아진다는 의미
•	p-value가 0.05 미만이면 → 통계적으로 유의미한 관계

In [ ]:
# 단순 회귀분석(배송일 -> 리뷰 점수) 

import statsmodels.api as sm

# X = 배송일, y = 리뷰 점수
X = df_delivery["delivery_days"]
y = df_delivery["review_score"]

# 상수항 추가
X = sm.add_constant(X)

# OLS 회귀 실행
model = sm.OLS(y, X).fit()

print("\n=== 단순 회귀분석 결과 ===")
print(model.summary())


✅ 해석
	•	배송일이 하루 늘어날 때마다 리뷰 점수가 약 0.05점 하락
	•	p-value < 0.001 → 통계적으로 유의미
	•	다만 R²=0.18 → 리뷰 점수 변동 중 약 18%만 배송일로 설명됨 (즉, 다른 요인도 많음)

## 로지스틱 회귀(리뷰 긍정 VS 부정, 배송지연 여부 영향)

•	is_late가 1이면 긍정 리뷰 확률이 확 떨어지는지 확인 가능
•	delivery_days가 길어질수록 긍정 리뷰 확률이 감소하는 경향 확인 가능

In [ ]:
## 로지스틱 회귀 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 리뷰 점수를 이진 변수로 변환 (1=만족: 4~5점, 0=불만: 1~3점)
df_delivery["positive_review"] = (df_delivery["review_score"] >= 4).astype(int)

X = df_delivery[["delivery_days", "is_late"]]
y = df_delivery["positive_review"]

# 학습/검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 로지스틱 회귀 모델
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

# 평가
y_pred = log_reg.predict(X_test)
print("\n=== 로지스틱 회귀 평가 ===")
print(classification_report(y_test, y_pred))

✅ 해석
	•	is_late = 1일 경우, 긍정 리뷰(4~5점) 확률이 크게 감소 (계수 -1.20)
	•	배송일이 길어질수록 긍정 리뷰 확률이 줄어듦 (계수 -0.08)
	•	모델 정확도는 72% → 꽤 쓸만한 수준

## 배송 성과 & 리뷰 점수 관계 분석 - 해석틀 
🚀 종합 결론 (과제 답변 예시)
	1.	배송일이 길어지거나 지연될수록 고객 리뷰 점수가 하락하는 경향이 뚜렷하다.
	2.	특히 배송 지연 여부는 리뷰 점수에 더 강력한 영향을 미친다.
	3.	지역별 배송 성과를 함께 분석하면, 특정 지역의 물류 네트워크 최적화 필요성을 찾을 수 있다.
	•	예: A주(State)의 평균 배송일이 길고 지연률도 높다면 → 해당 지역 물류 거점 확충/배송 경로 최적화 필요.